In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os 
from tqdm import tqdm
from tensorflow.keras import metrics


### Dataloader to load train and test data


In [9]:
## train test split 

In [10]:
class DataLoader(object):
    def __init__(self, root_dir, split_file):
        self.root_dir = root_dir
        self.split_file = split_file 
    def load_data(self):
        images_folder = os.path.join(self.root_dir,'images')
        train_test_split = os.path.join(root_dir, self.split_file)
        images_name_file = os.path.join(root_dir,'images.txt')

        with open(train_test_split,'r') as f:
            train_test_attrs = f.readlines()
        with open(images_name_file,'r') as f:
            images_name = f.readlines()        
        train_data = {}
        test_data = {}
        
        for k,image_name in enumerate(images_name):
            image_name = image_name.split(" ")[-1]
            image_label = image_name.split("/")[0]
            train_flag = int(train_test_attrs[k].split(" ")[-1])
            if train_flag == 1:
                if image_label not in train_data:
                    train_data[image_label]=[os.path.join(images_folder, image_name)]
                else:
                    train_data[image_label].append(os.path.join(images_folder, image_name))
            else:
                if image_label not in test_data:
                    test_data[image_label]=[os.path.join(images_folder, image_name)]
                else:
                    test_data[image_label].append(os.path.join(images_folder, image_name))   
        
        return train_data, test_data
    


In [11]:
root_dir = '/home/tabasherarif/misc/Untitled Folder/CUB_200_2011/CUB_200_2011/'
split_file = 'train_test_split.txt'
dataloader = DataLoader(root_dir, split_file)
train_data, test_data = dataloader.load_data()

### Features on pre-trained model
1. pretrained ResNet50 is loaded
2. classifier layers are excluded
3. pooling = avg, to average output tensor in the form of 2D instead of 4D
4. train data is passed through pre-trained model, features are stored as npy file 
5. test data is passed through pre-trained model, features are compared with stored train features 
6. smalled L1 norm between test & train features gives the closest match

In [12]:
# load pre-trained model 
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

2023-02-09 02:53:50.806139: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00MiB (rounded to 8388608)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-09 02:53:50.806177: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-02-09 02:53:50.806187: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 107, Chunks in use: 107. 26.8KiB allocated for chunks. 26.8KiB in use in bin. 17.7KiB client-requested in use in bin.
2023-02-09 02:53:50.806194: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 80, Chunks in use: 80. 40.2KiB allocated for chunks. 40.2KiB in use in bin. 40.0KiB client-requested in use in bin.
2023-02-09 02:53:50.806201: I tensorflow/tsl/framework/bfc

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

In [ ]:
# extract features of train set
labels = list(train_data.keys())
embeddings={}
for j in tqdm(range(len(labels))):
    label = labels[j]
    for image_path in train_data[label]:
        image_path=image_path.split("\n")[0]
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x,verbose=0)
        
        if label not in embeddings:
            embeddings[label]=[preds]
        else:
            embeddings[label].append(preds)
    
        
        

In [ ]:
# save features of train set
train_features_dir = '/home/tabasherarif/misc/Untitled Folder/train_features'
for label in labels:
    label_dir = os.path.join(train_features_dir, label)
    os.makedirs(name=label_dir, exist_ok=True)
    features = np.asarray(embeddings[label]).squeeze()
    feature_file = os.path.join(label_dir,label)
    np.save(feature_file, features)

In [ ]:
# compare features of train and test 
def compare_features(preds,stored_features):
    features_dirs = os.listdir(stored_features)
    min_val = 1000
    selected_label = ''
    for features_dir in features_dirs:
        features_dir_path = os.path.join(stored_features, features_dir,features_dir+'.npy')
        saved_features = np.load(features_dir_path)
        
        diff = saved_features - preds

        diff = np.linalg.norm(diff,axis=1)
        diff_min = np.mean(sorted(diff)[0:5])
        
        if diff_min < min_val:
            min_val = diff_min
            selected_label=features_dir
    
    return selected_label, min_val
         
        

In [ ]:
# get predictions on test set and make comparison with train set features 
stored_features = '/home/tabasherarif/misc/Untitled Folder/train_features/'
gt=[]
prediction=[]
for j in tqdm(range(len(labels))):
    label = labels[j]
    for image_path in test_data[label]:
        image_path=image_path.split("\n")[0]
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x,verbose=0)
        features_dir, min_val=compare_features(preds, stored_features)
        prediction.append(features_dir)
        gt.append(label)
    
    


In [ ]:
# accuracy 
correct = 0
for j in range(len(gt)):
    if gt[j] == prediction[j]:     
        correct += 1
accuracy = 100*correct/len(gt)
print (accuracy)    